In [1]:
from __future__ import print_function
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import pandas as pd
import numpy as np

import category_encoders as ce
import numpy as np
import pandas as pd
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
encoder = ce.OneHotEncoder()

df = pd.read_csv("students-all.csv").iloc[:,1:]
y = df.romantic
df = df.drop("romantic", axis =1)

encoder.fit(df)

OneHotEncoder(cols=['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob',
                    'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid',
                    'activities', 'nursery', 'higher', 'internet', 'major'],
              drop_invariant=False, handle_missing='value',
              handle_unknown='value', return_df=True, use_cat_names=False,
              verbose=0)

In [3]:
df_one_hot = encoder.transform(df)

In [4]:
df_one_hot.columns

Index(['school_1', 'school_2', 'sex_1', 'sex_2', 'age', 'address_1',
       'address_2', 'famsize_1', 'famsize_2', 'Pstatus_1', 'Pstatus_2', 'Medu',
       'Fedu', 'Mjob_1', 'Mjob_2', 'Mjob_3', 'Mjob_4', 'Mjob_5', 'Fjob_1',
       'Fjob_2', 'Fjob_3', 'Fjob_4', 'Fjob_5', 'reason_1', 'reason_2',
       'reason_3', 'reason_4', 'guardian_1', 'guardian_2', 'guardian_3',
       'traveltime', 'studytime', 'failures', 'schoolsup_1', 'schoolsup_2',
       'famsup_1', 'famsup_2', 'paid_1', 'paid_2', 'activities_1',
       'activities_2', 'nursery_1', 'nursery_2', 'higher_1', 'higher_2',
       'internet_1', 'internet_2', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'major_1', 'major_2'],
      dtype='object')

In [5]:
def parse(data, encoder):
    """
    Parsing data to the same format as in "students-all.csv"
    
    returns: nicks array and dataframe
    """
    nicks = np.array(data['Nick'])
    mails = np.array(data['Feedback mail'])
    data = data.iloc[:, 2:-1]

    column_names = [
        'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob',
        'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures',
        'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'internet',
        'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences',
        'G1', 'G2', 'G3'
    ]

    data.columns = column_names

    # deafults
    data['school'] = np.repeat("GP", data.shape[0])
    data['major'] = np.repeat("mat", data.shape[0])
    data['higher'] = np.repeat("yes", data.shape[0])

    # corect order
    data = data[[
        'school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu',
        'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime',
        'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities',
        'nursery', 'higher', 'internet', 'famrel', 'freetime', 'goout', 'Dalc',
        'Walc', 'health', 'absences', 'G1', 'G2', 'G3', 'major'
    ]]
    
    data['age'] = pd.to_numeric(data['age'])
    data['sex'] = np.where(data.sex == 'Male', 'M', 'F')
    data['address'] = np.where(data.address == 'Rural', 'R', 'U')
    data['famsize'] = np.where(data.famsize == 'more than 3', 'GT3', 'LE3')
    data['Pstatus'] = np.where(data.Pstatus == 'living apart', 'A', 'T')

    d1 = {
        'none': 0,
        'primary education': 1,
        'middle school': 2,
        'high school': 3,
        'higher education': 4
    }

    data['Medu'] = [d1[item] for item in list(data.Medu)]
    data['Fedu'] = [d1[item] for item in list(data.Fedu)]

    d2 = {
        'teacher': 'teacher',
        'healthcare': 'health',
        'civil services': 'civil',
        'home': 'at_home',
        'other': 'other'
    }

    data['Mjob'] = [d2[item] for item in list(data.Mjob)]
    data['Fjob'] = [d2[item] for item in list(data.Fjob)]

    d3 = {
        'close to home': 'home',
        'school reputation': 'reputation',
        'course preference': 'course',
        'other': 'other'
    }

    data['reason'] = [d3[item] for item in list(data.reason)]
    
    d4 = {'Mother':'mother', 'Father':'father', 'other':'other'}
    
    data['guardian'] = [d4[item] for item in list(data.guardian)]

    d5 = {'< 15':1, '15-30':2, '30-60':3,'60 >':4}
    
    data['traveltime'] = [d5[item] for item in list(data.traveltime)]
    
    d6 = {'< 2':1, '2-5':2, '5-10':3,'10 >':4}
    
    data['studytime'] = [d6[item] for item in list(data.studytime)]
    
    data['G1'] = np.array((pd.to_numeric(data.G1)/5)*20 , dtype = 'int64')
    data['G2'] = np.array((pd.to_numeric(data.G2)/5)*20 , dtype = 'int64') 
    data['G3'] = np.array((pd.to_numeric(data.G3)/5)*20 , dtype = 'int64')
    
    data['absences'] = np.array(pd.to_numeric(data.absences) , dtype = 'int64')
    data['failures'] = np.array(data.failures, dtype = 'int64')
    data['famrel']   = np.array(data.famrel, dtype = 'int64')
    data['freetime'] = np.array(data.freetime, dtype = 'int64')
    data['goout']    = np.array(data.goout, dtype = 'int64')
    data['Dalc']     = np.array(data.Dalc, dtype = 'int64')
    data['Walc']     = np.array(data.Walc, dtype = 'int64')
    data['health']   = np.array(data.health, dtype = 'int64')
    
    
    data_transformed = encoder.transform(data)
    
    return nicks, mails, data_transformed

In [6]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
creds = flow.run_local_server(port=0)




FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [ ]:
service = build('sheets', 'v4',credentials=creds)
sheet = service.spreadsheets()

In [ ]:
SAMPLE_SPREADSHEET_ID = '1e1tWLI0vD05bUj-wLWicOnl0iU-GWz0aaWEtRDlTQ2M'

cols = "AG"
rows = 4000

SAMPLE_RANGE_NAME = 'A1:'+ cols + str(rows)

In [ ]:
SAMPLE_RANGE_NAME

In [ ]:
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])
data = pd.DataFrame(values)
data.columns = data.iloc[0,:]
data = data.iloc[1:,:]
data.head()

In [ ]:
n, m, df_parsed = parse(data, encoder)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV

target = np.where(y=='yes', 1, 0)

X_train, X_test, y_train, y_test = train_test_split(df_one_hot,
                                                    target, test_size = 0.2, random_state = 666)

xgb_model = xgb.XGBClassifier(max_depth = 7,
                              booster = "dart",
                              colsample_bytree = 0.3,
                              learning_rate = 0.39,
                              reg_alpha = 0.9,
                              reg_lambda = 1.8,
                              subsample = 0.8)
xgb_model.fit(X_train,y_train)

#y_prob = xgb_model.predict_proba(X_test)[:,1]

xgb_model.predict_proba(X_test)[:,1]

In [ ]:
y_predict_prob

In [ ]:
xgb_model.predict_proba(df_parsed)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_predict_prob)